In [ ]:
!pip install -U clustimage

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 812 kB 12.0 MB/s 
     |████████████████████████████████| 88 kB 8.9 MB/s 
     |████████████████████████████████| 1.1 MB 40.6 MB/s 
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295206 sha256=8dd9e15346bd31290562977bb4d4ddaf1f1838f3a672b0ee5af8d08ebc6ca4f6
  Stored in directory: /root/.cache/pip/wheels/4c/d5/59/5e3e297533ddb09407769762985d134135064c6831e29a914e
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=c68fa56fff5cccd6a8f793761645792a5d7966b7574fe072d931a436f3266766
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=8bff9c8f1bb09b3d1bbc2fade05d0cf8016d370f1d41a9ee8d3a8ef4309153c6
  Stored in directory: /root/.cache/

In [ ]:
import cv2
import numpy as np
import time
from google.colab.patches import cv2_imshow
from google.colab import files
import matplotlib.pyplot as plt
from clustimage import Clustimage
import os 
from scipy import cluster
import shutil
from datetime import timedelta
import numpy as np

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import clustimage
print(clustimage.__version__)

1.5.9


In [ ]:
"""The tranformation from video into images and the cluster analyses was done using googledrive to store the data  """
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Keyframe Extraction

**Setup:** 

in function split_video change filename1 and filename2. Filename1 is the path where a folder for a video will be created. Filename2 will create a folder inside the filename1 folder where the frames will be stored. 

In the for-loop change video_path to the path where the shots from the shot detection step are stored.

In [ ]:
# i.e if video of duration 30 seconds, saves 10 frame per second = 300 frames saved in total
SAVING_FRAMES_PER_SECOND = 4

In [ ]:
def format_timedelta(td):
    """Utility function to format timedelta objects"""
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return result + ".00".replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":", "-")


def get_saving_frames_durations(cap, saving_fps):
    """A function that returns the list of durations where to save the frames"""
    s = []
    # get the clip duration by dividing number of frames by the number of frames per second
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
    # use np.arange() to make floating-point steps
    for i in np.arange(0, clip_duration, 1 / saving_fps):
        s.append(i)
    return s

In [ ]:
"""This function tranforms videos or the shots of videos into images based an the framerate per second.
To Setup the code on a different computer you need to change the path of filename1 and filename2 accordingly to your computer. Those folders will then be automatically created"""
def split_video(video_file, n, f): #n is the number of the video, f is the shotnumber of the video n     
    if n<10:
      filename1 = f"/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_0000{n}" #path to folder where the video number n will be stored 
      filename2 = f"/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_0000{n}/video_{f}" #path to folder where the shot f of video number n will be stored 
      # make a folder by the name of the video file
      if not os.path.isdir(filename1):
        os.mkdir(filename1)
      if not os.path.isdir(filename2):
        os.mkdir(filename2)
    elif  n>=10 and n<100:
      filename1 = f"/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_000{n}" #path to folder where the video number n will be stored 
      filename2 = f"/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_000{n}/video_{f}" #path to folder where the shot f of video number n will be stored 
      # make a folder by the name of the video file
      if not os.path.isdir(filename1):
        os.mkdir(filename1)
      if not os.path.isdir(filename2):
        os.mkdir(filename2)
    else:
      filename1 = f"/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_00{n}" #path to folder where the video number n will be stored 
      filename2 = f"/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_00{n}/video_{f}" #path to folder where the shot f of video number n will be stored 
      # make a folder by the name of the video file
      if not os.path.isdir(filename1):
        os.mkdir(filename1)
      if not os.path.isdir(filename2):
        os.mkdir(filename2)
    # read the video file    
    cap = cv2.VideoCapture(video_file)
    # get the FPS of the video
    fps = cap.get(cv2.CAP_PROP_FPS)
    # if the SAVING_FRAMES_PER_SECOND is above video FPS, then set it to FPS (as maximum)
    saving_frames_per_second = min(fps, SAVING_FRAMES_PER_SECOND)
    # get the list of duration spots to save
    saving_frames_durations = get_saving_frames_durations(cap, saving_frames_per_second)
    # start the loop
    count = 0
    while True:
        is_read, frame = cap.read()
        if not is_read:
            # break out of the loop if there are no frames to read
            break
        # get the duration by dividing the frame count by the FPS
        frame_duration = count / fps
        try:
            # get the earliest duration to save
            closest_duration = saving_frames_durations[0]
        except IndexError:
            # the list is empty, all duration frames were saved
            break
        if frame_duration >= closest_duration:
            # if closest duration is less than or equals the frame duration, 
            # then save the frame
            frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))
            cv2.imwrite(os.path.join(filename2, f"frame{frame_duration_formatted}.jpg"), frame) 
            # drop the duration spot from the list, since this duration spot is already saved
            try:
                saving_frames_durations.pop(0)
            except IndexError:
                pass
        # increment the frame count
        count += 1

In [ ]:
"""Loop goes through all videos and transforms the shots into images.
To Setup this on a different computer you need to change the video_path accordingly to where the shots of the videos are saved.
"""
for n in range(1,101): #n is the number of videos 
    if n<10:
        for f in range(0,700): #f is number of shots per video
          video_path = (f"/content/gdrive/MyDrive/Video_Splits/video_split_0000{n}/"+f"video_{f}"+ ".mp4") #path where the shots f of the video n are stores 
          try: 
            print(video_path)
            split_video(video_path, n, f)
          except:
            print(f"video_{f} doesn't excist")
            break
    elif n<100:
        for f in range(0,700):
          video_path = (f"/content/gdrive/MyDrive/Video_Splits/video_split_000{n}/"+f"video_{f}"+ ".mp4") #path where the shots f of the video n are stores 
          try: 
            print(video_path)
            split_video(video_path, n, f)
          except:
            print(f"video_{f} doesn't excist")
            break
    else:
        for f in range(0,700):
          video_path = (f"/content/gdrive/MyDrive/Video_Splits/video_split_00{n}/"+f"video_{f}"+ ".mp4") #path where the shots f of the video n are stores 
          try: 
            print(video_path)
            split_video(video_path, n, f)
          except:
            print(f"video_{f} doesn't excist")
            break
    
    

# Cluster Analysis

**Setup:**

in function cluster_analysis change filename1 and filename2. Filename1 is the path where a folder for a video will be created. Filename2 will create a folder inside the filename1 folder where the unique frames of a cluster will be stored. So inside the path of filename2 will be the output images of the 
clustering process which are the inputs for YOLO and AlexNet.

In the for-loop change the video_path to where frames of the keyframe extraction step are located. 

In [ ]:
"""This function takes images as an input does a agglomerative hierachical cluster analysis and gives out one keyframe image per cluster detected.
With min_cluster and max_cluster a range is determined on how many clusters there shall be per input.
To Setup the code on a different computer you need to change the path of filename1 and filename2 accordingly to your computer. Those folders will then be automatically created."""
def cluster_analysis(path, n, f): #path is the path to the folder with the input images, n is the number of the video and f is the number of shot in the video n  
  if n<10:
    filename1 = f"/content/gdrive/MyDrive/Cluster_Frames_Videos_Part1/video_split_0000{n}" #path to folder where the video n shall be stored
    filename2 = f"/content/gdrive/MyDrive/Cluster_Frames_Videos_Part1/video_split_0000{n}/video_{f}" #path to folder where the images of shot f of video n shall be stored 
    # make a folder by the name of the video file
    if not os.path.isdir(filename1):
      os.mkdir(filename1)
    if not os.path.isdir(filename2):
      os.mkdir(filename2)
  
    cl = Clustimage(method='pca')
    results = cl.fit_transform(path , min_clust=1, max_clust=10)
    cl.cluster(cluster='agglomerative', evaluate='dbindex', metric='euclidean', linkage='ward', min_clust=1, max_clust=10, cluster_space='low')
    unique_images = {}
    unique_images = cl.unique()
    
    #takes unique image per cluster and saves it 
    original = unique_images["pathnames"]
    for each in original:
      shutil.copy(each,filename2)

  elif n<100:
    filename1 = f"/content/gdrive/MyDrive/Cluster_Frames_Videos_Part1/video_split_000{n}" #path to folder where the video n shall be stored
    filename2 = f"/content/gdrive/MyDrive/Cluster_Frames_Videos_Part1/video_split_000{n}/video_{f}" #path to folder where the images of shot f of video n shall be stored 
    # make a folder by the name of the video file
    if not os.path.isdir(filename1):
      os.mkdir(filename1)
    if not os.path.isdir(filename2):
      os.mkdir(filename2)
    
    cl = Clustimage(method='pca')
    results = cl.fit_transform(path , min_clust=1, max_clust=10)
    cl.cluster(cluster='agglomerative', evaluate='dbindex', metric='euclidean', linkage='ward', min_clust=1, max_clust=10, cluster_space='low')
    unique_images = {}
    unique_images = cl.unique()
    
    #takes unique image per cluster and saves it 
    original = unique_images["pathnames"]
    for each in original:
      shutil.copy(each,filename2)
  else:
    filename1 = f"/content/gdrive/MyDrive/Cluster_Frames_Videos_Part1/video_split_00{n}" #path to folder where the video n shall be stored
    filename2 = f"/content/gdrive/MyDrive/Cluster_Frames_Videos_Part1/video_split_00{n}/video_{f}" #path to folder where the images of shot f of video n shall be stored 
    # make a folder by the name of the video file
    if not os.path.isdir(filename1):
      os.mkdir(filename1)
    if not os.path.isdir(filename2):
      os.mkdir(filename2)
    
    cl = Clustimage(method='pca')
    results = cl.fit_transform(path , min_clust=1, max_clust=10)
    cl.cluster(cluster='agglomerative', evaluate='dbindex', metric='euclidean', linkage='ward', min_clust=1, max_clust=10, cluster_space='low')
    unique_images = {}
    unique_images = cl.unique()
    
    #takes unique image per cluster and saves it    
    original = unique_images["pathnames"]
    for each in original:
      shutil.copy(each,filename2)


In [ ]:
"""Loop goes through all frames per shot and does cluster analysis on them.
To Setup this on a different computer you need to change the video_path accordingly to where the shots of the videos are saved."""
for n in range(1,101):
    if n<10:
        for f in range(0,700):
          video_path = (f"/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_0000{n}/"+f"video_{f}") #path to folder where the images of shot f of video n are stored 
          try: 
            print(video_path)
            cluster_analysis(video_path, n, f)
          except:
            print(f"video_{f} doesn't excist")
            break
    elif n<100:
        for f in range(0,700):
          video_path = (f"/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_000{n}/"+f"video_{f}") #path to folder where the images of shot f of video n are stored 
          print(video_path)
          try: 
            print(video_path)
            cluster_analysis(video_path, n, f)
          except:
            print(f"video_{f} doesn't excist")
            break
    else:
        for f in range(0,700):
          video_path = (f"/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_00{n}/"+f"video_{f}") #path to folder where the images of shot f of video n are stored 
          try: 
            print(video_path)
            cluster_analysis(video_path, n, f)
          except:
            print(f"video_{f} doesn't excist")
            break

[clustimage] >INFO> filepath is set to [/tmp/clustimage]
[clustimage] >INFO> filepath is set to [/tmp/clustimage]
[clustimage] >INFO> filepath is set to [/tmp/clustimage]
[clustimage] >INFO> Cleaning previous fitted model results
[clustimage] >INFO> Extracting images from: [/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_00044/video_0]
[clustimage] >INFO> [0] files are collected recursively from path: [/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_00044/video_0]
[clustimage] >INFO> [0] images are extracted.
[clustimage] >INFO> Reading and checking images.
[clustimage] >INFO> Extracting features using method: [pca]


/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_00044/video_0
/content/gdrive/MyDrive/Keyframes_Videos_Part_1/video_split_00044/video_0
video_0 doesn't excist
